In [1]:
import os

In [2]:
os.environ

environ{'SHELL': '/bin/bash',
        'CONDA_MKL_INTERFACE_LAYER_BACKUP': '',
        'CONDA_EXE': '/home/jhrag/anaconda3/bin/conda',
        '_CE_M': '',
        'AZURE_CONSTR_dasdata': 'DefaultEndpointsProtocol=https;AccountName=dasdata;AccountKey=X3Hx05zkPXvESvGzbS2fN2eIJt6/iclGM11BJK1S6CZ4XfIMX25>',
        'AZURE_KEY_lfhydrophone': 'UohL6H3jDaDLliug0g2Lg5qZZNKWdyxkq2S5KYLE0EFwLjo09auQp48PyhBRKJtYsfjXMmie/d/9+AStLFAniQ==',
        'PWD': '/home/jhrag',
        'LOGNAME': 'jhrag',
        'XDG_SESSION_TYPE': 'tty',
        'CONDA_PREFIX': '/home/jhrag/anaconda3/envs/ooi_dev',
        '_': '/home/jhrag/anaconda3/envs/ooi_dev/bin/jupyter',
        'AZURE_CONSTR_ooidata': 'DefaultEndpointsProtocol=https;AccountName=ooidata;AccountKey=ibLlKtCIS3BRkG2XveLGlTgs4dElF0PAs7HFKaZ5LhVuQiJgEhLC9uDLmz7QACPMr2qleg6QQTq8+AStSKqsqg==;EndpointSuffix=core.windows.net',
        'MOTD_SHOWN': 'pam',
        'HOME': '/home/jhrag',
        'LANG': 'C.UTF-8',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:m

# Play around with uniform mseed files...

In [1]:
import pandas as pd

In [5]:
for k in range(15):
    print(pd.Timestamp('2016-01-29') - pd.Timedelta(days=30*k))

2016-01-29 00:00:00
2015-12-30 00:00:00
2015-11-30 00:00:00
2015-10-31 00:00:00
2015-10-01 00:00:00
2015-09-01 00:00:00
2015-08-02 00:00:00
2015-07-03 00:00:00
2015-06-03 00:00:00
2015-05-04 00:00:00
2015-04-04 00:00:00
2015-03-05 00:00:00
2015-02-03 00:00:00
2015-01-04 00:00:00
2014-12-05 00:00:00


## Download from list of blobs

In [1]:
import os

In [2]:
os.environ['AZURE_CONSTR_ooidata']

'DefaultEndpointsProtocol=https;AccountName=ooidata;AccountKey=ibLlKtCIS3BRkG2XveLGlTgs4dElF0PAs7HFKaZ5LhVuQiJgEhLC9uDLmz7QACPMr2qleg6QQTq8+AStSKqsqg==;EndpointSuffix=core.windows.net'

In [4]:
from azure.storage.blob import BlobServiceClient

# Set the connection string and container name
connection_string = os.environ['AZURE_CONSTR_ooidata']
container_name = "lfhydrophonemseed"

# Create a BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a reference to the container
container_client = blob_service_client.get_container_client(container_name)

In [5]:
for blob in container_client:
    
    # Download the blob to the specified path
    with open('test.mseed', "wb") as download_file:
        download_stream = container_client.download_blob(blob)
        download_file.write(download_stream.readall())
        
    break

In [27]:
for blob in container_client.list_blobs():
    if blob.name == '2022_11_23.miniseed':
        # Set the download path to the current working directory
        download_path = 'test.mseed'

        # Download the blob to the specified path
        with open(download_path, "wb") as download_file:
            download_stream = container_client.download_blob(blob)
            download_file.write(download_stream.readall())

        print(f"Downloaded {blob.name} to {download_path}")

{'name': '2022_11_23.miniseed', 'container': 'lfhydrophonemseed', 'snapshot': None, 'version_id': None, 'is_current_version': None, 'blob_type': <BlobType.BLOCKBLOB: 'BlockBlob'>, 'metadata': {}, 'encrypted_metadata': None, 'last_modified': datetime.datetime(2023, 3, 13, 17, 50, 17, tzinfo=datetime.timezone.utc), 'etag': '0x8DB23EB6817862B', 'size': 3075930624, 'content_range': None, 'append_blob_committed_block_count': None, 'is_append_blob_sealed': None, 'page_blob_sequence_number': None, 'server_encrypted': True, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': 'application/octet-stream', 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'lease': {'status': 'unlocked', 'state': 'available', 'duration': None}, 'blob_tier': 'Hot', 'rehydrate_pr

In [26]:
blob.name

'2022_12_23.miniseed'

In [1]:
import obspy
import io
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
# open miniseed file and merge
reclen = 512
chunksize = 100000 * reclen # Around 50 MB
trs = []
    
print('openning mseed file...')
with io.open('test.mseed', "rb") as fh:
    while True:
        with io.BytesIO() as buf:
            c = fh.read(chunksize)
            if not c:
                break
            buf.write(c)
            buf.seek(0, 0)
            st = obspy.read(buf)

        for tr in st:
            trs.append(tr)
stream = obspy.core.stream.Stream(trs)

openning mseed file...


In [3]:
# force sampling rate to be exactly 200
for tr in stream:
    tr.stats.sampling_rate = 200
print('merging stream...')
# merge stream
merged_stream = stream.merge()

merging stream...


In [4]:
starttime = pd.Timestamp('2022-11-23T00:00:00.000000Z')
endtime = pd.Timestamp('2022-12-23T00:00:00.000000Z')

starttime = obspy.UTCDateTime(starttime)
endtime = obspy.UTCDateTime(endtime)

npts = int((endtime - starttime)*200 + 1)

station_names = ['AXBA1','AXCC1','AXEC2','HYSB1', 'HYS14']
calib_vals = [2257.53, 2480.98, 2421.9, 2311.11, 2499.09] # counts/PA
calib = dict(zip(station_names, calib_vals))

In [5]:
data = {}
print('constructing dataset...')
# Construct xarray dataset
for tr in merged_stream:
    # convert to float 32, fill masked values with nan, convert to Pa
    if isinstance(tr.data, np.ma.core.MaskedArray):
        data_single = tr.data.astype(np.float32)
        np.ma.set_fill_value(data_single, np.nan)
        data[tr.stats.station] = data_single.filled()
    elif isinstance(tr.data, np.ndarray):
        data[tr.stats.station] = tr.data.astype(np.float32)
    else:
        raise Exception(f'type of data {type(tr.data)}. expected numpy.ndarray or numpy.ma.core.MaskedArray')


    if (tr.stats.starttime == starttime) & (tr.stats.endtime == endtime):
        pass
    elif (tr.stats.starttime > starttime) | (tr.stats.endtime < endtime):
        # trace doesn't cover full expected range of data
        # need to pad nan values
        if tr.stats.starttime > starttime:
            npts_missing = round((tr.stats.starttime - starttime)*200)
            data[tr.stats.station] =  np.concatenate((np.ones(npts_missing)*np.nan, data[tr.stats.station]))
        if tr.stats.endtime < endtime:
            npts_missing = round((endtime - tr.stats.endtime)*200)
            data[tr.stats.station] = np.concatenate((data[tr.stats.station], np.ones(npts_missing)*np.nan))

        # Remove extra tick if endtime is 1 sample too long
        if (tr.stats.endtime == endtime + 0.005):
            data[tr.stats.station] = data[tr.stats.station][:-1]

    elif (tr.stats.endtime == endtime + 0.005):
        data[tr.stats.station] = data[tr.stats.station][:-1]
    else:
        raise Exception(f'current trace: {tr} not caught by starttime endtime cases')

    # check that data is the correct size before creating xr.Dataset
    if len(data[tr.stats.station]) != npts:
        print(npts)
        raise Exception(f'appending nan failed to give correct size for trace {tr}')

# Add NaNs for any missing stations
for station in station_names:
    try:
        data[station]
    except KeyError:
        data[station] = np.ones(npts)*np.NaN

constructing dataset...


In [15]:
# Convert data from numpy to xr.DataArray
for key in list(data.keys()):
    data[key] = xr.DataArray(data[key], dims=['time'])

# Calibrate data
for key in list(data.keys()):
    data[key] = data[key] / calib[key]

In [22]:
attrs = {
    'network':'OO',
    'channel':'HDH',
    'sampling_rate':200,
    'delta':0.005,
    'units':'Pa'
}

# create dataset
ds = xr.Dataset(data, attrs=attrs)
ds = ds.chunk({'time':3600*200})

In [23]:
ds

<xarray.Dataset>
Dimensions:  (time: 518400001)
Dimensions without coordinates: time
Data variables:
    AXBA1    (time) float32 dask.array<chunksize=(720000,), meta=np.ndarray>
    AXCC1    (time) float32 dask.array<chunksize=(720000,), meta=np.ndarray>
    AXEC2    (time) float32 dask.array<chunksize=(720000,), meta=np.ndarray>
    HYS14    (time) float32 dask.array<chunksize=(720000,), meta=np.ndarray>
    HYSB1    (time) float64 dask.array<chunksize=(720000,), meta=np.ndarray>
Attributes:
    network:        OO
    channel:        HDH
    sampling_rate:  200
    delta:          0.005
    units:          Pa

In [24]:
ds.to_zarr('/datadrive/lfhydrophone/2023_11_23_2023_12_23.zarr')